In [47]:
import pandas as pd
import numpy as np
import nltk
from sklearn.metrics import accuracy_score
from neo4j import GraphDatabase, basic_auth


In [2]:
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [3]:
folder_path = "/home/ganesh/Documents/NLP-Fraunhofer/KG/tasks_1-20_v1-2/en/"
filename = "qa1_single-supporting-fact_train.txt"
data_qa1 = pd.read_csv(folder_path+filename, delimiter="\t", names=['sentence', 'answer', 'factid'])
data_qa1 = data_qa1.fillna("")
data_qa1[:6]

,sentence,answer,factid
0,1 Mary moved to the bathroom.,,
1,2 John went to the hallway.,,
2,3 Where is Mary?,bathroom,1
3,4 Daniel went back to the hallway.,,
4,5 Sandra moved to the garden.,,
5,6 Where is Daniel?,hallway,4


In [4]:
#tag the sentence
tag_sentence = lambda row: 'S' if row.answer == '' else 'Q'
data_qa1['type'] = data_qa1.apply(tag_sentence, axis=1)

tokenize = lambda row: nltk.word_tokenize(row.sentence)[1:]
data_qa1.sentence = data_qa1.apply(tokenize, axis=1)

data_qa1 = data_qa1.drop("factid", axis=1)
data_qa1[:6]

,sentence,answer,type
0,"[Mary, moved, to, the, bathroom, .]",,S
1,"[John, went, to, the, hallway, .]",,S
2,"[Where, is, Mary, ?]",bathroom,Q
3,"[Daniel, went, back, to, the, hallway, .]",,S
4,"[Sandra, moved, to, the, garden, .]",,S
5,"[Where, is, Daniel, ?]",hallway,Q


In [38]:
def statements(df):
    return df[df.type == "S"].reset_index(drop=True).drop("answer",axis=1).drop("type",axis=1)


def questions(df):
    return df[df.type == "Q"].reset_index(drop=True).drop("answer",axis=1).drop("type",axis=1)

def extract_statment(tags):
    sub, rel, obj = '', '', ''
    for word,tag in tags:
        if tag == 'NNP':
            sub = word
        elif tag == 'VBD' or word == 'journeyed':
            rel = word
        if tag == 'NNP' or tag== 'NN':
            obj = word
    return (sub, rel, obj)

def extract_question(tags):
    eud = ''
    for word,tag in tags:
        if tag == 'NNP' or tag== 'NN':
            eud = word
    return eud

def extract(row):
    if row.type =='S':
        return extract_statment(row.tag)
    else:
        return extract_question(row.tag)
    

In [6]:
statements(data_qa1)[:4]

,sentence
0,"[Mary, moved, to, the, bathroom, .]"
1,"[John, went, to, the, hallway, .]"
2,"[Daniel, went, back, to, the, hallway, .]"
3,"[Sandra, moved, to, the, garden, .]"


In [7]:
questions(data_qa1)[:4]

,sentence
0,"[Where, is, Mary, ?]"
1,"[Where, is, Daniel, ?]"
2,"[Where, is, Daniel, ?]"
3,"[Where, is, Daniel, ?]"


In [8]:
pos_tag = lambda row: nltk.pos_tag(row.sentence)
data_qa1["tag"] = data_qa1.apply(pos_tag, axis=1)

In [9]:
data_qa1[:6]

,sentence,answer,type,tag
0,"[Mary, moved, to, the, bathroom, .]",,S,"[(Mary, NNP), (moved, VBD), (to, TO), (the, DT..."
1,"[John, went, to, the, hallway, .]",,S,"[(John, NNP), (went, VBD), (to, TO), (the, DT)..."
2,"[Where, is, Mary, ?]",bathroom,Q,"[(Where, WRB), (is, VBZ), (Mary, NNP), (?, .)]"
3,"[Daniel, went, back, to, the, hallway, .]",,S,"[(Daniel, NNP), (went, VBD), (back, RB), (to, ..."
4,"[Sandra, moved, to, the, garden, .]",,S,"[(Sandra, NNP), (moved, VBD), (to, TO), (the, ..."
5,"[Where, is, Daniel, ?]",hallway,Q,"[(Where, WRB), (is, VBZ), (Daniel, NNP), (?, .)]"


In [39]:
data_qa1['extracted']=data_qa1.apply(extract, axis=1)
data_qa1[:6]

,sentence,answer,type,tag,extracted
0,"[Mary, moved, to, the, bathroom, .]",,S,"[(Mary, NNP), (moved, VBD), (to, TO), (the, DT...","(Mary, moved, bathroom)"
1,"[John, went, to, the, hallway, .]",,S,"[(John, NNP), (went, VBD), (to, TO), (the, DT)...","(John, went, hallway)"
2,"[Where, is, Mary, ?]",bathroom,Q,"[(Where, WRB), (is, VBZ), (Mary, NNP), (?, .)]",Mary
3,"[Daniel, went, back, to, the, hallway, .]",,S,"[(Daniel, NNP), (went, VBD), (back, RB), (to, ...","(Daniel, went, hallway)"
4,"[Sandra, moved, to, the, garden, .]",,S,"[(Sandra, NNP), (moved, VBD), (to, TO), (the, ...","(Sandra, moved, garden)"
5,"[Where, is, Daniel, ?]",hallway,Q,"[(Where, WRB), (is, VBZ), (Daniel, NNP), (?, .)]",Daniel


## Debug function

In [45]:
def person_statements(person):
    stat = statements(data_qa1)
    return stat[stat.extracted.map(lambda t: t[0]==person)]

def person_statemnets_recent(person, n=5):
    return person_statements(person)[-n:].iloc[::-1]

In [44]:
person_statements('Sandra')[:3]

,sentence,tag,extracted
3,"[Sandra, moved, to, the, garden, .]","[(Sandra, NNP), (moved, VBD), (to, TO), (the, ...","(Sandra, moved, garden)"
5,"[Sandra, journeyed, to, the, bathroom, .]","[(Sandra, NNP), (journeyed, VBD), (to, TO), (t...","(Sandra, journeyed, bathroom)"
10,"[Sandra, travelled, to, the, office, .]","[(Sandra, NNP), (travelled, VBD), (to, TO), (t...","(Sandra, travelled, office)"


In [46]:
person_statemnets_recent('Daniel', n=3)

,sentence,tag,extracted
1999,"[Daniel, went, to, the, garden, .]","[(Daniel, NNP), (went, VBD), (to, TO), (the, D...","(Daniel, went, garden)"
1996,"[Daniel, travelled, to, the, kitchen, .]","[(Daniel, NNP), (travelled, VBD), (to, TO), (t...","(Daniel, travelled, kitchen)"
1992,"[Daniel, moved, to, the, office, .]","[(Daniel, NNP), (moved, VBD), (to, TO), (the, ...","(Daniel, moved, office)"


## Build Graph

In [48]:
# Database Credentials
uri             = "bolt://localhost:7687"
userName        = "neo4j"
password        = " "

In [51]:
driver  = GraphDatabase.driver(uri, auth=(userName, password)) 
# graph = Graph(host='localhost', user=userName, password=password)
# tx = graph.begin()

In [64]:
#This will delete all the data/node in the database
def reset_db():
    session = driver.session()
    session.run("Match (n) Detach Delete n")
    
def create(query, n=0):
    session = driver.session()
    stat = statements(data_qa1)
    n= len(stat) if n <=0 else n
    for sub, rel, obj in stat[:n].extracted:
        session.run(query, subject=sub, relation=rel, obj=obj)

In [62]:
reset_db()

In [65]:
v1_query = '''
    MERGE (s:SUBJECT {name: $subject})
    MERGE (o:OBJECT {name: $obj})  
    MERGE (s)-[r:RELATION {name: $relation}]->(o)
'''
create(v1_query)